In [1]:
import os

while os.getcwd().split("/")[-1] != "alfa-hack-rag":
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
import pandas as pd

In [6]:
reference_df = pd.read_csv("ranking_results/submit_1.csv")
test_df = pd.read_csv("ranking_results/submit2025-11-13 21:11:55.455071.csv")

In [15]:
def parse_web_list(web_list_series):
    return web_list_series.str[1:-1].str.split(", ").apply(lambda x: [int(num) for num in x])

In [ ]:
reference_df["web_list"] = parse_web_list(reference_df["web_list"])
test_df["web_list"] = parse_web_list(test_df["web_list"])

In [25]:
def calculate_hit_at_k(reference_df, test_df, k=5):
    """
    Рассчитывает метрику Hit@k и возвращает DataFrame с детальными результатами

    Parameters:
    reference_df: DataFrame с эталонными данными
    test_df: DataFrame с предсказанными данными
    k: количество top результатов для проверки

    Returns:
    tuple: (hit_at_k, results_df) где results_df содержит детальную информацию по каждому вопросу
    """
    
    results = []
    valid_questions = 0
    total_hits = 0

    # Проходим по всем вопросам в тестовом DataFrame
    for _, test_row in test_df.iterrows():
        q_id = test_row["q_id"]
        test_webs = test_row["web_list"][:k]  # Берем первые k элементов

        # Находим соответствующий вопрос в эталонном DataFrame
        reference_row = reference_df[reference_df["q_id"] == q_id]

        # Если вопрос есть в эталонных данных
        if not reference_row.empty:
            reference_webs = reference_row.iloc[0]["web_list"]

            # Проверяем, есть ли хотя бы одно пересечение
            intersection = set(test_webs) & set(reference_webs)
            hit = 1 if intersection else 0
            
            total_hits += hit
            valid_questions += 1

            # Сохраняем детальную информацию
            results.append({
                'q_id': q_id,
                'reference_webs': reference_webs,
                'test_webs': test_webs,
                'intersection': list(intersection),
                'intersection_count': len(intersection),
                'hit@k': hit
            })

    # Создаем DataFrame с результатами
    results_df = pd.DataFrame(results)
    
    # Рассчитываем итоговую метрику
    if valid_questions > 0:
        hit_at_k = total_hits / valid_questions
    else:
        hit_at_k = 0

    return hit_at_k, results_df

# Пример использования:
hit_at_5, results_df = calculate_hit_at_k(reference_df, test_df, k=5)
print(f"Hit@5 = {hit_at_5:.4f}")
results_df

Hit@5 = 0.4007


,q_id,reference_webs,test_webs,intersection,intersection_count,hit@k
0,1,"[1404, 1252, 501, 1567, 844]","[1567, 1896, 595, 599, 590]",[1567],1,1
1,2,"[1098, 1607, 372, 1659, 1080]","[1567, 1896, 595, 1517, 590]",[],0,0
2,3,"[856, 116, 1724, 114, 1604]","[1567, 16, 1204, 1573, 790]",[],0,0
3,4,"[1043, 1680, 1590, 1924, 478]","[1204, 1573, 468, 647, 1806]",[],0,0
4,5,"[927, 1029, 1909, 1193, 1033]","[1193, 1029, 1033, 1034, 1026]","[1193, 1029, 1033]",3,1
...,...,...,...,...,...,...
6972,6973,"[420, 1516, 1552, 1541, 1201]","[420, 1902, 1320, 1552, 1900]","[1552, 420]",2,1
6973,6974,"[1213, 837, 368, 697, 1587]","[61, 1896, 62, 1573, 1567]",[],0,0
6974,6975,"[1592, 1929, 688, 1590, 1680]","[796, 1043, 1204, 468, 37]",[],0,0
6975,6976,"[1567, 1896, 1884, 1554, 1532]","[1896, 1567, 37, 61, 1742]","[1896, 1567]",2,1
